In [1]:
import numpy as np
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn import datasets, metrics
from sklearn.metrics import accuracy_score
import time
import pandas as pd

In [2]:
# 定义 KMeans 类
class KMeans(BaseEstimator, ClassifierMixin):
    def __init__(self, k):
        """
        KMeans 类的初始化函数

        Parameters:
        - k: 簇的数量

        Attributes:
        - k: 簇的数量
        - x: 输入数据
        - y: 标签数据（未使用）
        - labels: 簇分配的标签
        - centers: 簇中心
        - iterations: 迭代次数，默认为500
        """
        self.k = k
        self.x = None
        self.y = None
        self.labels = None
        self.centers = None
        self.iterations = 500

    def quick_L2(self, x, a):
        """
        计算两组数据点之间的欧氏距离的平方
    
        Parameters:
        - x: 第一组数据点
        - a: 第二组数据点
    
        Returns:
        - dis: 欧氏距离的平方
        """
        # 计算两组数据点的内积
        dis = -2 * np.dot(x, a.T)
        
        # 计算第一组数据点的平方和，使用einsum函数对元素进行逐个相乘并按行求和
        dis += np.einsum('ij,ij->i', x, x)[:, np.newaxis]
        
        # 计算第二组数据点的平方和，使用einsum函数对元素进行逐个相乘并按行求和
        dis += np.einsum('ij,ij->i', a, a)[np.newaxis, :]
        
        return dis


    def fit(self, x, y=None, init_method='random_point', seed=None, eps=1e-5):
        """
        拟合（训练）K均值模型

        Parameters:
        - x: 输入数据
        - y: 标签数据（未使用）
        - init_method: 初始化方法，'random_point'表示随机选择数据点作为初始簇中心，其他值表示使用随机数初始化
        - seed: 随机数种子
        - eps: 收敛阈值

        Returns:
        - None
        """
        self.x = x
        self.y = y
        self.centers = 0

        if seed is not None:
            np.random.seed(seed)
        if init_method == 'random_point':
            # 随机选择数据点作为初始簇中心
            self.centers = x[np.random.choice(x.shape[0], self.k), :]
        else:
            # 使用随机数初始化
            self.centers = np.random.randint(np.min(x), np.max(x), (x.shape[0], self.k))

        pre_centers = self.centers.copy()
        for i in range(self.iterations):
            # 计算欧氏距离
            dis = self.quick_L2(self.x, self.centers)
            # 分配数据点到最近的簇
            idx = np.argmin(dis, axis=1)
            # 更新簇中心
            for j in range(self.centers.shape[0]):
                self.centers[j, :] = np.mean(self.x[idx == j, :], axis=0)
            # 判断是否收敛
            if np.mean(np.abs(pre_centers - self.centers)) < eps:
                break
            pre_centers = self.centers.copy()

    def predict(self, a=None):
        """
        预测数据点属于哪个簇

        Parameters:
        - a: 输入数据，默认为训练数据

        Returns:
        - idx: 数据点所属的簇的标签
        """
        if a is None:
            a = self.x
        dis = self.quick_L2(a, self.centers)
        idx = np.argmin(dis, axis=1)
        return idx

In [3]:
# 主程序
if __name__ == '__main__':
    # 加载鸢尾花数据集
    iris = datasets.load_iris()
    data = iris['data']
    labels = iris['target']

In [4]:
    t = time.perf_counter()
    # 创建 KMeans 类的实例，设置簇的数量为3
    kmeans = KMeans(k=3)

    # 拟合模型
    kmeans.fit(data)

    # 预测数据点所属的簇
    res = kmeans.predict()

    # 打印预测结果和执行时间
    print('iris-predict:')
    print(res)

iris-predict:
[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 0 1 1 1 1
 1 1 0 0 1 1 1 1 0 1 0 1 0 1 1 0 0 1 1 1 1 1 0 1 1 1 1 0 1 1 1 0 1 1 1 0 1
 1 0]


In [5]:
    from sklearn.metrics import adjusted_rand_score
    ari = adjusted_rand_score(labels, res)
    print('Adjusted Rand Index (ARI) in iris: ', ari)

Adjusted Rand Index (ARI) in iris:  0.7302382722834697


In [6]:
    print('iris-SC指标: ' + str(metrics.silhouette_score(data, res, metric='euclidean')))

iris-SC指标: 0.5528190123564095


In [7]:
    # 读取名为 'sonar.all-data.csv' 的 CSV 文件，无表头，使用逗号作为分隔符
    sonar = pd.read_csv('sonar.all-data.csv', header=None, sep=',')
    
    # 从 sonar 数据中选择前 208 行和前 60 列的子集，存储为 sonar1
    sonar1 = sonar.iloc[0:208, 0:60]
    
    # 将 sonar1 转换为 NumPy 数组
    data2 = np.array(sonar1)
    
    # 创建 labels2 数组，其中 R 类别对应标签值为 1，M 类别对应标签值为 0
    labels2 = np.zeros(208)
    labels2[sonar.iloc[:, 60] == 'R'] = 1

In [8]:
    labels2

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0.])

In [9]:
    # 创建 KMeans 对象 kmeans2，设置簇的数量为 2
    kmeans2 = KMeans(k=2)
    
    # 拟合模型，使用前 208 行 60 列的数据
    kmeans2.fit(data2)
    
    # 预测数据点所属的簇
    res2 = kmeans2.predict()
    
    # 打印预测结果
    print('sonar-predict:')
    print(res2)

sonar-predict:
[0 1 1 0 1 1 1 1 1 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 0
 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 1 1 1 1 0 0 0 1 0 0 0 1
 1 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 0 0 0 0 0]


In [10]:
    ari2 = adjusted_rand_score(labels2, res2)
    print('Adjusted Rand Index (ARI) in sonar: ', ari2)

Adjusted Rand Index (ARI) in sonar:  0.010869872549936507


In [11]:
    # 计算并打印 SC（轮廓系数）指标，使用欧氏距离作为度量方式
    print('sonar-SC指标: ' + str(metrics.silhouette_score(data2, res2, metric='euclidean')))

sonar-SC指标: 0.19896316739419573
